<a href="https://colab.research.google.com/github/MAL3X-01/DS-Unit-2-Tree-Ensembles/blob/master/Final_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Libraries

In [2]:
!pip install -U pandas-profiling

     |████████████████████████████████| 133kB 4.9MB/s 
     |████████████████████████████████| 614kB 16.9MB/s 
     |████████████████████████████████| 225kB 39.2MB/s 
     |████████████████████████████████| 768kB 38.5MB/s 
     |████████████████████████████████| 194kB 43.4MB/s 
     |████████████████████████████████| 51kB 18.5MB/s 
     |████████████████████████████████| 51kB 16.6MB/s 
     |████████████████████████████████| 737kB 41.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/45/10/b5/5565b02c204a3bb87490a22e0e4c382ea06337bb79fd74d58a
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Stored in directory: /root/.cache/pip/wheels/b0/b2/96/2074eee7dbf7b7df69d004c9b6ac4e32dad04fb7666cf943bd
Successfully built pandas-profiling htmlmin confuse
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1

In [1]:
!pip install pdpbox category_encoders

     |████████████████████████████████| 57.7MB 1.3MB/s 
     |████████████████████████████████| 92kB 17.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/7d/08/51/63fd122b04a2c87d780464eeffb94867c75bd96a64d500a3fe
Successfully built pdpbox


# Importing Kaggle Competition Data

In [66]:
%matplotlib inline
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

LOCAL = '../data/tanzania/'
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/tanzania/'

train = pd.merge(pd.read_csv(WEB + 'train_features.csv'), 
                 pd.read_csv(WEB + 'train_labels.csv'))
test = pd.read_csv(WEB + 'test_features.csv')
sample_submission = pd.read_csv(WEB + 'sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

# Data Exploration

In [67]:
train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [68]:
train.describe(exclude='number')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6,3
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625,32259


In [0]:
import pandas_profiling
#train.profile_report()

# Feature Engineering

In [0]:
def wrangle(X):
    """Wrangles train, validate, and test sets in the same way"""
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    X['construct_avail'] = X['construction_year'].apply(lambda x: 1 if x > 1500 
                                                        else 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace them with the column mean.
    cols_with_zeros = ['construction_year', 'longitude', 'latitude', 'amount_tsh', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract year from date_recorded
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['week_recorded'] = X['date_recorded'].dt.week
    # quantity & quantity_group are duplicates, so drop one
    X = X.drop(columns='quantity_group')
    
    # for categoricals with missing values, fill with the category 'MISSING'
    #categoricals = X.select_dtypes(exclude='number').columns
    #for col in categoricals:
    #    X[col] = X[col].fillna('MISSING')
    
    return X


train = wrangle(train)
test = wrangle(test)

In [0]:
cols_ = ['construction_year', 'longitude', 'latitude', 'amount_tsh', 'population']
def wrangle2(X):
  for col in cols_:
    X[col] = X[col].fillna(pd.concat([train, test])[col].mean())
    
  X['water_per_person'] = X['amount_tsh']/ X['population']  
  return X

In [72]:
train = wrangle2(train)
test = wrangle2(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [73]:
train['construction_year'].value_counts().head()

1996.857033    20709
2010.000000     2645
2008.000000     2613
2009.000000     2533
2000.000000     2091
Name: construction_year, dtype: int64

In [74]:
train['year_recorded'].value_counts()

2011    28674
2013    24271
2012     6424
2004       30
2002        1
Name: year_recorded, dtype: int64

In [0]:
train['construction_year'] = train['construction_year'].apply(lambda x: int(x))
test['construction_year'] = test['construction_year'].apply(lambda x: int(x))

In [0]:
train['age'] = (train['year_recorded'] - train['construction_year']) 
test['age'] = (test['year_recorded'] - test['construction_year']) 

In [0]:
train['age'] = train['age'].apply(lambda x : np.NaN if x<=0 else x)
test['age'] = test['age'].apply(lambda x : np.NaN if x<=0 else x)

In [78]:
train['age'] = train['age'].fillna(round(pd.concat([train,test])['age'].mean()))
test['age'] = test['age'].fillna(round(pd.concat([train,test])['age'].mean()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [0]:
train['gps_height'] = train['gps_height'].apply(lambda x : np.NaN if x==0 else x)
test['gps_height'] = test['gps_height'].apply(lambda x : np.NaN if x==0 else x)

In [80]:
train['gps_height'] = train['gps_height'].fillna(round(pd.concat([train,test])['gps_height'].median()))
test['gps_height'] = test['gps_height'].fillna(round(pd.concat([train,test])['gps_height'].median()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


# Train Test Split / Selecting Features

In [81]:
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=5)

train.shape, val.shape, test.shape

((47520, 46), (11880, 46), (14358, 45))

#### Select features

In [0]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target & id
train_features = train.drop(columns=[target, 'id', 'num_private', 'wpt_name',
                                    'recorded_by','subvillage'])
test = test.drop(columns=['id', 'num_private', 'wpt_name',
                                    'recorded_by','subvillage'])
# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()
#categorical_features = ['quantity', 'waterpoint_type', 'extraction_type', 'water_quality',
                        #'payment_type', 'management', 'region', 'source']

# Combine the lists 
features = numeric_features + categorical_features

# Pipeline/RFC

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer


In [0]:
# 0.9199284511784511

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

rfc = RandomForestClassifier(n_estimators=250, min_samples_split=4, max_depth = 25,
                             criterion='gini', max_features='auto', oob_score=True,
                             random_state=5, n_jobs=-1)

encoder = ce.OrdinalEncoder()

scaler = MinMaxScaler()

pipe = Pipeline(steps=[('encoder', encoder), ('scaler', scaler), ('rfc', rfc)])

pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)

In [21]:
# 0.8128787878787879
from sklearn.metrics import accuracy_score
y_pred = pipe.predict(X_val)
accuracy_score(y_pred, y_val)

0.8076879910213244

# Boosting

## GradientBoosting

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [24]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]


GBC = GradientBoostingClassifier(max_depth= 5, random_state=40, n_estimators=100)

encoder = ce.OrdinalEncoder()

scaler = StandardScaler()

pipe2 = Pipeline(steps=[('encoder', encoder), ('scaler', scaler), ('GBC', GBC)])

pipe2.fit(X_train, y_train)
pipe2.score(X_train, y_train)

0.7993506493506494

In [25]:
from sklearn.metrics import accuracy_score
y_pred = pipe2.predict(X_val)
accuracy_score(y_pred, y_val)

0.7803030303030303

## XGBoost

In [103]:
#0.9778018278018278
# 0.9781144781144782
#0.9805765993265994
from xgboost import XGBClassifier

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

xgb = XGBClassifier(n_estimators=300, max_depth=10, learning_rate=0.6, subsample=0.7)

encoder = ce.OrdinalEncoder()

scaler = StandardScaler()

pipe3 = Pipeline(steps=[('encoder', encoder), ('scaler', scaler), ('xgb', xgb)])

pipe3.fit(X_train, y_train)
pipe3.score(X_train, y_train)

0.996969696969697

In [104]:
# 0.805611672278339
# 0.8072390572390572
# 0.8095959595959596
# 0.8101010101010101
from sklearn.metrics import accuracy_score
y_pred = pipe3.predict(X_val)
accuracy_score(y_pred, y_val)

0.7973905723905724

# Submission

In [0]:
y_pred = pipe3.predict(X_test)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-17.csv', index=False)

In [40]:
!head submission-14.csv

id,status_group
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional
52449,functional
24806,non functional
28965,non functional
36301,non functional
